In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt
from cytoolz import dissoc

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenbake as s
import fov
import fluid
from geometry import Cell, Round

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

In [ ]:
def print_fovs(fov_dims, metadata):
    for fov_name, fov_dim in fov_dims.items():
        fovs, unit_cell_height, active_height = fov.fovs_per_chip(fov_dim, **metadata)
        print(f"{fov_name}: {fov_dim[0]} x {fov_dim[1]}")
        # print(metadata)
        print("    FOVs: {} ({} x {})".format(np.product(fovs), fovs[0], fovs[1]))
        print("    unit cell height:", unit_cell_height)
        print(
            "    active height: {} (margin: {})".format(
                active_height, fov_dim[1] - active_height
            )
        )

In [ ]:
camera_dims = {
    "iris": (4.25, np.array([5056, 2960])),
    "bsi": (6.5, np.array([2048, 2048])),
}
fov_dims = {}
for camera, (pixel_size, fov_dim) in camera_dims.items():
    for mag in (20, 40, 60, 100):
        fov_dims[f"{camera} {mag}x"] = fov_dim * pixel_size / mag

In [ ]:
def print_flow_info(height, Q_per_snake, mu, nu, metadata):
    for name, md in metadata.items():
        R_, extra = fluid.snake_resistance(
            height, md, length_unit=u.um, return_extra=True
        )
        R = R_ * mu
        # print('R', R.to(u.mbar * u.min / u.uL))
        delta_P = R * Q_per_snake
        print(f"{name}: {delta_P.to(u.bar)}")
        if "flow_nonuniformity" in extra:
            print(f"    flow nonuniformity: {extra['flow_nonuniformity']}")
        if "manifold_width" in md:
            L_e = fluid.entrance_length(
                height,
                md["manifold_width"] * u.um,
                Q_per_snake * md["manifold_split"],
                nu,
            )
            print(f"    entrance length: {L_e}")

# Designs

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([34e3, 18e3])
chip_dims0 = np.array([23e3, 13e3])
chip_dims1 = np.array([40e3, 22e3])

In [ ]:
plt.figure(figsize=(10, 5))
ax = plt.gca()
ax.add_patch(plt.Rectangle(-coverslip_dims / 2, *coverslip_dims, fill=False))
ax.add_patch(plt.Rectangle(-chip_dims0 / 2, *chip_dims0, fill=False, ls="-."))
ax.add_patch(plt.Rectangle(-chip_dims1 / 2, *chip_dims1, fill=False, ls="--"))
ax.add_patch(plt.Rectangle(-chip_dims / 2, *chip_dims, fill=False))
ax.set_xlim(-1.2 * coverslip_dims[0] / 2, 1.2 * coverslip_dims[0] / 2)
ax.set_ylim(-1.2 * coverslip_dims[1] / 2, 1.2 * coverslip_dims[1] / 2)
ax.set_aspect("equal")

In [ ]:
%%time
metadata = {}
base_params = dict(
    design_func=s.snake,
    dims=chip_dims,
    split=8,
    gap_lanes=0,
    trench_length=35,
    # feeding_channel_width=40,
    horizontal_margin=2e3,
    trench_gap=20,
    trench_width=1.4,
    # trench_spacing=2.1,
    port_margin=0.6e3,
    top_margin=1.2e3,
    bottom_margin=0.6e3,
    draw_trenches=True,
    metadata=metadata,
)
manifold_params = dissoc(
    {
        **base_params,
        **dict(
            design_func=s.manifold_snake,
            dims=chip_dims,
            split=None,
            num_manifolds=1,
            manifold_width=200,
            manifold_input_margin=2e3,
            manifold_bend_margin=0.2e3,
            manifold_bend_radius=300,
            manifold_margin=100,
            feeding_channel_width=40,
            port_margin=0.5e3,
            top_margin=0.9e3,
            bottom_margin=0.9e3,
            # trench_spacing=2.1,
        ),
    },
    "horizontal_margin",
    "gap_lanes",
)
params = [
    dict(
        feeding_channel_width=80,
        trench_spacing=1.1,
        registration_marks=True,
        chip_id=0,
        mark_size=2,
        mark_spacing=1.5,
        tick_period=50,
        **base_params,
    ),
    dict(
        feeding_channel_width=80,
        trench_spacing=2.1,
        registration_marks=True,
        chip_id=0,
        mark_size=1,
        mark_spacing=1,
        tick_period=50,
        **base_params,
    ),
    dict(
        feeding_channel_width=80,
        trench_spacing=3.1,
        registration_marks=False,
        **base_params,
    ),
    dict(
        lanes_per_snake=3,
        trench_spacing=2.1,
        registration_marks=True,
        chip_id=1,
        mark_size=1,
        mark_spacing=1,
        **manifold_params,
    ),
    dict(
        lanes_per_snake=3,
        trench_spacing=2.1,
        manifold_input_style="u-turn",
        registration_marks=False,
        **manifold_params,
    ),
    dict(
        lanes_per_snake=5,
        trench_spacing=2.1,
        registration_marks=True,
        chip_id=1,
        mark_size=1,
        mark_spacing=1,
        **manifold_params,
    ),
]
chip_names = [
    "Basilisk {ports} FC{feeding_channel_width} L{trench_length} W{trench_width} TS{trench_spacing}{reg}".format(
        ports=f"S{p['split']}"
        if np.isscalar(p["split"])
        else "S{} LS{}".format(p["num_manifolds"], p["lanes_per_snake"]),
        reg=f" M{p['mark_size']}" if p["registration_marks"] else "",
        **p,
    )
    for p in params
]
chips = [s.chip(name, **p) for p, name in zip(params, chip_names)]
wafer_manifest = "\n".join([f"{idx+1}) " + name for idx, name in enumerate(chip_names)])
main_cell = s.wafer(
    chips,
    text_right="Basilisk \n JQS/DE/YG 200226",
    text_left=wafer_manifest,
    text=True,
    mask=False,
    chip_dims=chip_dims,
    diameter=(4 * u.inch).to(u.um).magnitude,
)
s.write_gds(main_cell, "200226basilisk.gds")

In [ ]:
print(wafer_manifest)

In [ ]:
{k: v["num_trenches"] for k, v in metadata.items()}

In [ ]:
Q_per_snake = 20 * u.uL / u.min
height = 75 * u.um
mu = 0.7 * u.cP  # dynamic viscosity
nu = mu / (1 * u.g / u.cm**3)  # kinematic viscosity
print_flow_info(height, Q_per_snake, mu, nu, metadata)

In [ ]:
(
    Q_per_snake
    * mu
    * fluid.snake_resistance(
        80 * u.um,
        {"snake_length": [17920 * 19], "feeding_channel_width": 90},
        length_unit=u.um,
    )
).to(u.bar)

In [ ]:
(
    Q_per_snake
    * mu
    * fluid.snake_resistance(
        80 * u.um,
        {"snake_length": [17920 * 25], "feeding_channel_width": 45},
        length_unit=u.um,
    )
).to(u.bar)

In [ ]:
print_fovs(fov_dims, metadata["Basilisk S8 FC80 L35 W1.4 TS2.1 M1"])

In [ ]:
print_fovs(fov_dims, metadata["Basilisk S1 LS3 FC40 L35 W1.4 TS2.1 M1"])